In [ ]:
import torch
import torch.nn as nn

class d2l(): # 请忽视，只是为了去除代码块中的错误信息提示，若运行请正确导入包
    pass

class AttentionDecoder(): # 此处仅仅是去除代码块中的错误信息提示，实际运行请在d2l中导入
    pass

def masked_softmax():
    pass

RUN=False # 代码没有数据处理，没法正常运行，需要修改代码
if (not RUN):
    raise NotImplementedError()

In [ ]:
# predict
# 在书中，predict流程是通用的
#@save
def predict_seq2seq(net, src_sentence, src_vocab, tgt_vocab, num_steps,
                    device, save_attention_weights=False):
    """序列到序列模型的预测"""
    ### 在预测时将net设置为评估模式
    net.eval()
    
    # 原句子预处理
    src_tokens = src_vocab[src_sentence.lower().split(' ')] + [
        src_vocab['<eos>']]
    enc_valid_len = torch.tensor([len(src_tokens)], device=device)
    src_tokens = d2l.truncate_pad(src_tokens, num_steps, src_vocab['<pad>'])
    ### 添加批量轴
    enc_X = torch.unsqueeze(
        torch.tensor(src_tokens, dtype=torch.long, device=device), dim=0)
    
    # encoding
    enc_outputs = net.encoder(enc_X, enc_valid_len)
    
    
    dec_state = net.decoder.init_state(enc_outputs, enc_valid_len)
    ### 添加批量轴
    dec_X = torch.unsqueeze(torch.tensor(
        [tgt_vocab['<bos>']], dtype=torch.long, device=device), dim=0)
    output_seq, attention_weight_seq = [], []
    for _ in range(num_steps):
        Y, dec_state = net.decoder(dec_X, dec_state)
        ### 我们使用具有预测最高可能性的词元，作为解码器在下一时间步的输入
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        ### 保存注意力权重（稍后讨论）
        if save_attention_weights:
            attention_weight_seq.append(net.decoder.attention_weights)
        ### 一旦序列结束词元被预测，输出序列的生成就完成了
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq)), attention_weight_seq

### 参数说明 / 次要的忽略
|参数|type|描述|
|-|-|-|
|net|d2l.EncoderDecoder||
|src_sentence|str|初始句子，新预测用于完善句子|
|src_vocab|d2l.?|词表，用于将str句子转为数字索引 - tokenization|
|tgt_vocab|d2l.?|target vocab|
|num_steps|int?|具体含义/用途待定|

### 运行
1. 原句子预处理
    
    - str句子小写，按空格分离，转为token，追加&lt;pad&gt;token
    - 填充&lt;pad&gt;或截断，确保src_tokens长度为设定的num_steps 形状：(num_steps,)
    - 添加batch维度，形状(1, num_steps) **预测中的batch_size默认为1**


2. encoding
  
    输出
    - output (num_steps,batch_size,num_hiddens) 包含每个batch的GRU最后一层的在每个时间步的隐藏状态
    - state (num_layers,batch_size,num_hiddens) 包含最后一时间步每个batch每层的隐藏状态

    note:**隐藏状态可以理解为包含上下文信息(state)，也可以理解为包含预测的下一个词的概率(decoder在预测时传入上一个token，得到的output中包含下一词的信息，通过一个全连接层转化)**

3. 预测
    
    - 初始化解码器状态(dec_state,会循环更新多次使用)
    - 初始化编码器传入的词为&lt;bos&gt; (变量名dec_X)
    - 进行num_steps次循环，直到生成的词为&lt;eos&gt;
    - (循环)上一个词dec_X和解码器状态传入decoder，得到预测最高可能性的词元(1, 1, vocab_size)并更新解码器状态，用于下一次循环
    - (循环)取得最大可能的词元，更新dec_X用于下一次预测；同时取得相应的token，加入最终结果

In [ ]:
# encoding
#@save
class Seq2SeqEncoder(d2l.Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          dropout=dropout)

    def forward(self, X, *args):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X)
        # 在循环神经网络模型中，第一个轴对应于时间步
        X = X.permute(1, 0, 2)
        # 如果未提及状态，则默认为0
        output, state = self.rnn(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

### forward说明
#### X形状
|操作|形状|
|-|-|
|传参|(batch_size, num_steps)|
|embedding|(batch_size, num_steps, embed_size)|
|交换维度，适配rnn输入|(num_steps, batch_size, embed_size)|

#### rnn输出参数说明 / 参考pytorch文档
|符号|含义|
|-|-|
|N|batch_size|
|L|序列长度 num_steps|
|D|与双向GRU有关，双向为2，单向为1|
|$H_in$|输入大小(GRU初始化时传入的是embed_size)|
|$H_out$|隐藏层大小(GRU初始化时传入的是num_hiddens)|

|文档参数|代码|描述|
|-|-|-|
|output|output|对于非批处理输入，其张量形状为 $(L,D∗H_out)$；当 'batch_first=False'(代码默认为'False') 时，形状为 $(L,N,D∗H_out)$；当 'batch_first=True'(可忽略) 时，形状为 $(N,L,D∗H_out)$，其中包含'GRU'最后一层在每个时间步't'的输出特征'(h_t)'。如果输入是'torch.nn.utils.rnn.PackedSequence'，则输出也将是打包序列。<br /><br />也就是说，代码中output形状为(num_steps, batch_size, num_hiddens)，包含每个batch的GRU最后一层的在每个时间步的隐藏状态(h_t)|
|h_n|state|张量形状为$(D∗num_layers, H_out)$或$(D∗num_layers, N, H_out)$，包含输入序列的最终隐藏状态。<br /><br />也就是说，代码中state形状为(num_layers, batch_size, num_hiddens)，包含最后一时间步每个batch每层的隐藏状态|

In [ ]:
# decoder + attention
class Seq2SeqAttentionDecoder(AttentionDecoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqAttentionDecoder, self).__init__(**kwargs)
        self.attention = d2l.AdditiveAttention(
            num_hiddens, num_hiddens, num_hiddens, dropout)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(
            embed_size + num_hiddens, num_hiddens, num_layers,
            dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        # outputs的形状为(batch_size，num_steps，num_hiddens).
        # hidden_state的形状为(num_layers，batch_size，num_hiddens)
        outputs, hidden_state = enc_outputs
        return (outputs.permute(1, 0, 2), hidden_state, enc_valid_lens)

    def forward(self, X, state):
        # enc_outputs的形状为(batch_size,num_steps,num_hiddens).
        # hidden_state的形状为(num_layers,batch_size,
        # num_hiddens)
        enc_outputs, hidden_state, enc_valid_lens = state
        # 输出X的形状为(num_steps,batch_size,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        outputs, self._attention_weights = [], []
        for x in X:
            # query的形状为(batch_size,1,num_hiddens)
            query = torch.unsqueeze(hidden_state[-1], dim=1)
            # context的形状为(batch_size,1,num_hiddens)
            context = self.attention(
                query, enc_outputs, enc_outputs, enc_valid_lens)
            # 在特征维度上连结
            x = torch.cat((context, torch.unsqueeze(x, dim=1)), dim=-1)
            # 将x变形为(1,batch_size,embed_size+num_hiddens)
            out, hidden_state = self.rnn(x.permute(1, 0, 2), hidden_state)
            outputs.append(out)
            self._attention_weights.append(self.attention.attention_weights)
        # 全连接层变换后，outputs的形状为
        # (num_steps,batch_size,vocab_size)
        outputs = self.dense(torch.cat(outputs, dim=0))
        return outputs.permute(1, 0, 2), [enc_outputs, hidden_state,
                                          enc_valid_lens]

    @property
    def attention_weights(self):
        return self._attention_weights

### init_state
encoder的返回值传入其中，得到用于decoder.forward的state

state:
- enc_outputs (batch_size, num_steps, num_hiddens)
- hidden_state (num_layers, batch_size, num_hiddens)
- enc_valid_lens

### forward
#### 传参
X：(batch_size, num_steps, embed_size) 预测中，batch_size=1 num_steps=1，训练中未知

#### 流程(注意x的大小写)
- 将X的时间步维度提前 (num_steps,batch_size,embed_size) (要逐个时间步处理数据)
- (为什么要这样做可能与训练机制有关)逐个时间步提取出x(batch_size, embed_size) - X所有批次的指定时间步的词的embedding结果
- (循环中)通过attention机制计算出上下文，与x拼接
  + 使用最后一时间步最后一层的状态作为query(添加了维度，去适配attention的要求输入) query: (batch_size, 1, num_hiddens)
  + 对GRU最后一层每个时间步的状态(同时作为key,value)进行查询 key/value: (batch_size, num_steps, num_hiddens) -> context: (batch_size, 1, num_hiddens)
  + 为x增加维度，与context形状适配。context(前)与x拼接 -> x: (batch_size, 1, num_hiddens + embed_size)
- (循环中)x交换维度，适配GRU，传入其中，得到out(1, batch_size, num_hiddens), hidden_state(num_layers, batch_size, num_hiddens)(可供下一轮使用)。out会累计到outputs(循环完成: (num_steps, batch_size, num_hiddens))里
- 通过全连接层转化outputs，(看到形状里面的vocab_size你大概可以猜到是什么目的了) 得到(num_steps,batch_size,vocab_size)
- 返回outputs(batch_size,num_steps,vocab_size)(交换了维度), dec_state(解码器状态)((batch_size, num_steps, num_hiddens), (num_layers, batch_size, num_hiddens), enc_valid_lens)

In [ ]:
# attention
#@save
class AdditiveAttention(nn.Module):
    """加性注意力"""
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens):
        queries, keys = self.W_q(queries), self.W_k(keys)
        # 在维度扩展后，
        # queries的形状：(batch_size，查询的个数，1，num_hidden)
        # key的形状：(batch_size，1，“键－值”对的个数，num_hiddens)
        # 使用广播方式进行求和
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        features = torch.tanh(features)
        # self.w_v仅有一个输出，因此从形状中移除最后那个维度。
        # scores的形状：(batch_size，查询的个数，“键-值”对的个数)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        # values的形状：(batch_size，“键－值”对的个数，值的维度)
        return torch.bmm(self.dropout(self.attention_weights), values)